In [48]:
import pandas as pd
import numpy as np
from datetime import datetime, timezone
from tqdm import tqdm
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
# 导入docplex
from docplex.mp.model import Model
# 创建模型对象

data_anly= pd.read_csv('data/data_cal.csv')
data_anly

,Datetime,Measured & Upscaled,Most recent forecast,Monitored capacity,ShiftMeasured,Fluctuation15M,PredFluctuation15M,ShiftFluctuation15M
0,2022-01-01 00:00:00,1803.48,1881.52,2254.4,1803.48,0.00,78.04,0.00
1,2022-01-01 00:15:00,1962.39,1855.50,2254.4,1803.48,158.91,52.02,0.00
2,2022-01-01 00:30:00,1801.63,1823.16,2254.4,1962.39,-160.76,-139.23,158.91
3,2022-01-01 00:45:00,1824.41,1795.96,2254.4,1801.63,22.78,-5.67,-160.76
4,2022-01-01 01:00:00,1661.57,1779.28,2254.4,1824.41,-162.84,-45.13,22.78
...,...,...,...,...,...,...,...,...
35035,2022-12-31 22:45:00,2187.05,2133.89,2254.4,2177.33,9.72,-43.44,16.48
35036,2022-12-31 23:00:00,2177.68,2123.49,2254.4,2187.05,-9.37,-63.56,9.72
35037,2022-12-31 23:15:00,2180.85,2129.58,2254.4,2177.68,3.17,-48.10,-9.37
35038,2022-12-31 23:30:00,2177.87,2147.49,2254.4,2180.85,-2.98,-33.36,3.17


In [49]:
param = {'ratio':0.1,
         'C_C':0.1*2254.4,
         'C_W':2254.4, # MW
         'SOE_init':0.5*225.44,
         'SOE_max':0.95*225.44,
         'SOE_min':0.05*225.44,
         'P_BSS_max':225.44,
         'P_BSS_min':-225.44,
         'PT10M':1/4,
         'refine_power':50,
         'slight_upward':6,# 向上波动四分位数
         'slight_downward':-7, # 向下波动四分位数
         'condition1':"upward_fluctuation",
         'condition2':"downward_fluctuation",
         'condition3':"upward_slight_fluctuation",
         'condition4':"downward_slight_fluctuation",
         'condition5':"no_fluctuation",
         'nita_c': 0.932,
         'nita_d': 0.932}


In [50]:

def solver(P_W_list,n,M,mode,cap,power):
    # 函数功能：求解最佳储能功率
    # 函数输入: P_W_list风机功率列表，n为总步数，M为窗口宽度
    # 函数输出：P_BSS_rolling储能滚动优化出力功率列表,E_rolling储能滚动优化能量状态列表
    P_BSS_rolling = []
    E_rolling = []
    real_steps = n - M
    P_optimize = P_W_list

    # 添加约束
    for real_step in tqdm(range(0, real_steps), desc='Optimizing'):
        # print(f"第{real_step+1}次滚动求解")
        sub_list = P_W_list[real_step:real_step+M]

        # 新建求解模型
        cplex_obj = Model()

        # 限制变量范围
        P_BSS = [cplex_obj.continuous_var(name='P_BSS[%d]' % i, lb=-power, ub=power) for i in range(M)]# P_c[i]意味着在第i步的充电功率 # 充放电功率的标签范围是0~n-1
        z_c = [cplex_obj.binary_var(name='z_c[%d]' % i) for i in range(M)]# P_c[i]意味着在第i步的充电功率 # 充放电功率的标签范围是0~n-1
        z_d = [cplex_obj.binary_var(name='z_d[%d]' % i) for i in range(M)]# P_c[i]意味着在第i步的充电功率 # 充放电功率的标签范围是0~n-1
        P_BSS_plus = [cplex_obj.continuous_var(name='P_BSS_plus[%d]' % i, lb = 0, ub = power) for i in range(M)]# P_c[i]意味着在第i步的充电功率 # 充放电功率的标签范围是0~n-1
        P_BSS_minus = [cplex_obj.continuous_var(name='P_BSS_minus[%d]' % i, lb = 0, ub = power) for i in range(M)]# P_c[i]意味着在第i步的充电功率 # 充放电功率的标签范围是0~n-1

        # 创建由于从0到n-1有n个能量状态，所以有n-1条状态传递方程，还有对0状态的等式约束
        
        E = [cplex_obj.continuous_var(name='E[%d]' % i, lb=0.05*cap, ub=0.95*cap) for i in range(M)]
        
        # 确保 P_BSS_plus 和 P_BSS_minus 正确表示 P_BSS 的正负部分
        for i in range(M): # param['P_BSS_max']
            cplex_obj.add_constraint(P_BSS[i] == P_BSS_plus[i] - P_BSS_minus[i])
            cplex_obj.add_constraint(P_BSS_plus[i] <= power * z_c[i])  # 当 z_c[i]=0 时，P_BSS_plus 必须为 0
            cplex_obj.add_constraint(P_BSS_minus[i] <= power * z_d[i])  # 当 z_d[i]=0 时，P_BSS_minus 必须为 0
            cplex_obj.add_constraint(z_c[i] + z_d[i] <= 1)  # 确保不同时为正负
            
        # 添加初始状态，以等式约束形式
        if real_step == 0:
            cplex_obj.add_constraint(E[0] == 0.5*cap) # ,param['SOE_init']
            cplex_obj.add_constraint(P_BSS_plus[0] == 0) 
            cplex_obj.add_constraint(P_BSS_minus[0] == 0) 
            last_power = P_W_list[0]

        else:
            cplex_obj.add_constraint(E[0] == last_state[1] - P_BSS_plus[0]/0.932*(1/4) + P_BSS_minus[0]*0.932*(1/4))

        # 等式约束
        for l in range(0,M-1):
            cplex_obj.add_constraint(E[l+1] == E[l] - P_BSS_plus[l+1]/0.932*(1/4) + P_BSS_minus[l+1]*0.932*(1/4)) # 电能量+是充电-是放电

        # 计算波动
        fluctuation = [((sub_list[i] + P_BSS_plus[i] - P_BSS_minus[i] ) - 
                (last_power if i == 0 else sub_list[i-1] + P_BSS_plus[i-1]- P_BSS_minus[i-1]))
               for i in range(0, M)]
        E_fluctuation = [(E[i]-E[i-1])for i in range(0, M)]
        E_half_fluctuation = [(E[i]-0.5*cap)for i in range(0, M)]
        # 计算波动的绝对值，并将其相加，作为目标函数
        if mode == 'square':
            objective_expr = cplex_obj.sum([fluctuation[i] * fluctuation[i]  for i in range(len(fluctuation))])
        elif mode == 'abs':
            objective_expr = cplex_obj.sum([cplex_obj.abs(fluctuation[i])  for i in range(len(fluctuation))])
        elif mode == 'range':
            objective_expr = cplex_obj.sum([cplex_obj.max( cplex_obj.abs(fluctuation[i])- 50,0)  for i in range(len(fluctuation))])
        elif mode == 'abs_with_E':
            objective_expr = cplex_obj.sum([0.7*cplex_obj.abs(fluctuation[i])+0.3*cplex_obj.abs(E_fluctuation[i])  for i in range(len(fluctuation))])
            # objective_expr = cplex_obj.sum([cplex_obj.max( cplex_obj.abs(fluctuation[i])- 50,0)  for i in range(len(fluctuation))])
        elif mode == 'abs_with_E0.5':
            objective_expr = cplex_obj.sum([0.7*cplex_obj.abs(fluctuation[i])+0.3*cplex_obj.abs(E_half_fluctuation[i])  for i in range(len(fluctuation))])
        elif mode == 'range_with_E0.5':
            objective_expr = cplex_obj.sum([cplex_obj.max( cplex_obj.abs(fluctuation[i])- 50,0) +0.3*cplex_obj.abs(E_half_fluctuation[i]) for i in range(len(fluctuation))])

        # 最大化目标函数
        cplex_obj.minimize(objective_expr)

        # cplex_obj.print_information()

        # 求解优化问题
        solution = cplex_obj.solve()
        # 获取结果
        if solution:
            # print(f"最优值为：{cplex_obj.objective_value:.2f}")
            p_bss_values = [round(P_BSS[i].solution_value,2) for i in range(M)]
            
            E_values = [round(E[i].solution_value,2) for i in range(M)]
            last_power = P_W_list[real_step] + P_BSS[0].solution_value
            last_state = [p_bss_values[0],E_values[0],last_power]

            P_optimize[real_step] = round(last_power,2)

            P_BSS_rolling.append(round(p_bss_values[0],2))
            E_rolling.append(round(E_values[0],2))
            # print(f'子序列为：{([P_optimize[0]] if real_step==0 else [P_optimize[real_step-1]])   +sub_list}')
            # print(f"P_BSS的取值为：{p_bss_values}")
            # print(f"E的取值为：{E_values}")
            # print()
        else:
            print("求解失败")
    
    return P_BSS_rolling,E_rolling

In [51]:
import pandas as pd
def generate_analyse_dataframe(datetime_list,P_list,p_bss_values,E_values):
    # 函数功能：将数据列表转化为分析的数据表
    # 函数输入：时间数据列（优化长度）、原功率数据列（优化长度）、储能功率数据列（优化长度）、能量数据列（优化长度）
    # 列表转换为DataFrame
    df_test = pd.DataFrame({
        'Datetime':datetime_list,
        'P_W_list': P_list,
        'p_bss_values': p_bss_values,
        'E':E_values
    })
    df_test['Datetime'] = pd.to_datetime(df_test['Datetime'])
    df_test['P_W_shift'] = df_test['P_W_list'].shift(1)
    df_test['P_W_shift'] = df_test['P_W_shift'].fillna(df_test['P_W_list'].iloc[0])# df_test['P_W_shift'].iloc[0]
    df_test['original_flucuation'] = df_test['P_W_list'] - df_test['P_W_shift']

    df_test['P_CW'] = df_test['P_W_list'] + df_test['p_bss_values']
    df_test['P_CW_shift'] = df_test['P_CW'].shift(1)
    df_test['P_CW_shift'] = df_test['P_CW_shift'].fillna(df_test['P_CW'].iloc[0])
    df_test['after_flucuation'] = df_test['P_CW'] - df_test['P_CW_shift']

    return df_test



In [52]:
# 对运行结果数据集进行分析
def analyse(df):
    df_test = df.copy()
    df_test['abs_original_flucuation'] = df_test['original_flucuation'].abs()
    df_test['abs_after_flucuation'] = df_test['after_flucuation'].abs()

    # 使用 apply 函数对元素进行条件判断和赋值
    df_test['original_above'] = df_test['abs_original_flucuation'].apply(lambda x: x - 50 if x > 50 else 0)
    df_test['after_above'] = df_test['abs_after_flucuation'].apply(lambda x: x - 50 if x > 50 else 0)

    original_total_fluctuation = df_test['original_above'].sum()
    after_total_fluctuation = df_test['after_above'].sum()

    df_test['original_above_label'] = df_test['original_above'].apply(lambda x: 1 if x > 0 else 0)
    df_test['after_above_label'] = df_test['after_above'].apply(lambda x: 1 if x > 0 else 0)
    original_above_points = df_test['original_above_label'].sum()
    after_above_points = df_test['after_above_label'].sum()

    print(f'原波动率越限值求和值是{original_total_fluctuation}')
    print(f'优化后的波动率越限值求和值是{after_total_fluctuation}')
    print(f'原波动率越限点数是{original_above_points}')
    print(f'优化后的波动率越限点数求和值是{after_above_points}')
    
    return original_total_fluctuation,after_total_fluctuation,original_above_points,after_above_points

In [53]:
def gather_param_anly(param_name,param_list,original_fluc_rate_list,after_fluc_rate_list,original_fluc_points_list,after_fluc_points_list):
    param_anly_df = pd.DataFrame({
        f'{param_name}':param_list,
        'original_fluc_rate':original_fluc_rate_list,
        'after_fluc_rate': after_fluc_rate_list,
        'original_fluc_points': original_fluc_points_list,
        'after_fluc_points':after_fluc_points_list
    })
    return param_anly_df

In [54]:
def optimize_by_date(df,datestr,M): # 修改输入数据集为往后延M个点，保证一天是动96个点
    data_date = df.copy()
    data_date['Datetime'] = pd.to_datetime(data_date['Datetime'])
    target_date = pd.to_datetime(datestr).date()

    # 选取指定日期数据集
    target_date_df = data_date[data_date['Datetime'].dt.date == pd.to_datetime(target_date).date()]
    # 选取指定日期及往后 M 个点的数据集
    extra_df = data_date[data_date['Datetime'].dt.date > target_date][0:M]
    # 合并两个数据集
    target_df = pd.concat([target_date_df, extra_df], axis=0)

    return target_df

##  设计选择窗格、容量大小、功率大小的文件名字
- 日期（先尝试日周期会不会出现明显效果）
- 文件夹：日期
- 文件名：date{}_width{}_cap{}_power{}_.csv


In [55]:
df = data_anly.copy()
datestr_list = ['2022-02-02','2022-06-19','2022-09-18','2022-12-07']
cap = 0.3*2254.4
power = 0.3*2254.4
method_list = ['abs','range','range','range_with_E0.5']
width_list = [30,1,30,30]

for date_name in  datestr_list:
    print(f"日期为{date_name}")
    total_anly_route = f'ch5_data/analyse_by_date/{date_name}/{date_name}_total_anly.csv'
    param_list = []
    original_fluc_rate_list = []
    after_fluc_rate_list = []
    original_fluc_points_list = []
    after_fluc_points_list = []

    for i in range(len(method_list)):
        method = method_list[i]
        M = width_list[i]
        print(f"方法为{method}")
        param_list.append(M)
        print(f"窗口长度为{M}")

        result_file_route = f'ch5_data/analyse_by_date/{date_name}/{date_name}_{method}_{M}_anly.csv'  

        target_df = optimize_by_date(df,date_name,M)

        n = len(target_df)
        P_W_list = 	list(target_df['Measured & Upscaled']) # 获取solver的输入
        p_bss_values,E_values = solver(P_W_list,n,M,method,cap,power) 

        datetime_list = list(target_df['Datetime'][:96]) 
        P_W_list_original = list(target_df['Measured & Upscaled'][:96])
        df_test = generate_analyse_dataframe(datetime_list,P_W_list_original,p_bss_values,E_values)
        original_total_fluctuation,after_total_fluctuation,original_above_points,after_above_points = analyse(df_test)

        original_fluc_rate_list.append(original_total_fluctuation)
        after_fluc_rate_list.append(after_total_fluctuation)
        original_fluc_points_list.append(original_above_points)
        after_fluc_points_list.append(after_above_points)
        df_test.to_csv(result_file_route)

    param_list = method_list

    gather_param_df = gather_param_anly(method,param_list,original_fluc_rate_list,after_fluc_rate_list,original_fluc_points_list,after_fluc_points_list)
    gather_param_df.to_csv(total_anly_route)




日期为2022-02-02
方法为abs
窗口长度为30


Optimizing: 100%|██████████| 96/96 [00:43<00:00,  2.23it/s]


原波动率越限值求和值是6081.0599999999995
优化后的波动率越限值求和值是1969.4300000000003
原波动率越限点数是57
优化后的波动率越限点数求和值是17
方法为range
窗口长度为1


Optimizing: 100%|██████████| 96/96 [00:01<00:00, 50.93it/s]


原波动率越限值求和值是6081.0599999999995
优化后的波动率越限值求和值是3784.390000000001
原波动率越限点数是57
优化后的波动率越限点数求和值是30
方法为range
窗口长度为30


...

原波动率越限值求和值是7302.94
优化后的波动率越限值求和值是1132.1400000000012
原波动率越限点数是60
优化后的波动率越限点数求和值是20
方法为range
窗口长度为30


Optimizing: 100%|██████████| 96/96 [00:08<00:00, 11.54it/s]


原波动率越限值求和值是7302.94
优化后的波动率越限值求和值是1.2505552149377763e-12
原波动率越限点数是60
优化后的波动率越限点数求和值是11
方法为range_with_E0.5
窗口长度为30


Optimizing: 100%|██████████| 96/96 [03:28<00:00,  2.17s/it]

原波动率越限值求和值是7302.94
优化后的波动率越限值求和值是1.0231815394945443e-12
原波动率越限点数是60
优化后的波动率越限点数求和值是8
